In [ ]:
#Choose "TEST" or "VALIDATION" to define which dataset you would like to use to work with model
#"TEST" for 501 images
#"VALIDATTION" for 20 images

TEST_OR_VALIDATION = "VALIDATION"



---


### Import TensorFlow and helper libraries


---



In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2



---


### Upload **Trash_Image_Classification** repository to Colab

Upload = clone whole repository which contains zipped datasets (for this notebook only test images or validation images will be used) and saved model (.h5).

After unzipping, the datasets should be available from **/content** folder:
* ***train*** folder - */content/dataset_trash/train*
* ***test*** folder - */content/dataset_trash/test*
* ***validation*** folder - */content/dataset_trash/validation* (these images model has seen neither during training nor testing)

Saved model should be available from:
* */content/Trash_Image_Classification/trash_image_classification.h5*


---



In [ ]:
isdir = os.path.isdir('/content/Trash_Image_Classification') #checking if repo is already present in Colab

if isdir == True:
  pass #do nothing
else:
  !git clone https://github.com/marcin-ch/Trash_Image_Classification.git
  !unzip /content/Trash_Image_Classification/dataset_trash.zip



---


### Create arrays with images and labels


---



In [ ]:
#TEST IMAGES

if TEST_OR_VALIDATION == "TEST":
  path = '/content/dataset_trash/test'
elif TEST_OR_VALIDATION == "VALIDATION":
  path = '/content/dataset_trash/validation'
else:
  print('* * * * * "TEST_OR_VALIDATION" is not valid! Please check again. * * * * *')

test_images = []
test_labels = []
shape = (32,32)

for i in os.listdir(path):
        img = cv2.imread(os.path.join(path,i))
        if i.split('.')[1] == 'jpg':
            #print(i)
            img2 = cv2.resize(img,shape)
            img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
            test_images.append(img2)
            test_labels.append(i.split('_')[0])



---


### Convert **images** to Numpy and normalize values to be between 0 and 1


---



In [ ]:
test_images = np.array(test_images)
print("TEST shape", test_images.shape)

test_images = test_images / 255.0 #Normalize pixel values to be between 0 and 1



---


### Convert **labels** to Numpy and change names of trash (strings) to coresponding integers

* `cardboard` -> `0`
* `glass` -> `1`
* `metal` -> `2`
* `paper` -> `3`
* `plastic` -> `4`
* `trash` -> `5`


---



In [ ]:
print("\n* * * * * * TEST LABELS * * * * *")
#print(test_labels)
test_labels = np.array(test_labels)
test_labels[test_labels == "cardboard"] = 0
test_labels[test_labels == "glass"] = 1
test_labels[test_labels == "metal"] = 2
test_labels[test_labels == "paper"] = 3
test_labels[test_labels == "plastic"] = 4
test_labels[test_labels == "trash"] = 5
print("Data type of the array test_labels is:", test_labels.dtype)

test_labels = test_labels.astype(int)
print("New type of the array test_labels is: ", test_labels.dtype)

print("Shape of test_labels is: ", test_labels.shape)
#print(test_labels)



---


### Define the class names


---



In [ ]:
class_names = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash'] #corresponds with cardboard->0, glass->1, metal->2, paper->3, plastic->4, trash->5



---


### Load the saved model


---



In [ ]:
loaded_model = tf.keras.models.load_model('/content/Trash_Image_Classification/trash_image_classification.h5')

loaded_model.summary()



---


### Make predictions


---



In [ ]:
predictions = loaded_model.predict(test_images)



---


### Verify predictions
https://www.tensorflow.org/tutorials/keras/classification#verify_predictions

Correct prediction labels are blue and incorrect prediction labels are red. The number gives the percentage (out of 100) for the predicted label.

---



In [ ]:
#Helper functions for drawing graphs

def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(True) #MCH
  #plt.grid(False)
  plt.xticks(range(6), class_names, rotation=75, fontsize=12) #MCH
  #plt.xticks(range(10))
  #plt.yticks([]) #MCH Passing an empty list removes all yticks
  thisplot = plt.bar(range(6), predictions_array*100, color="#777777")
  plt.ylim([0, 100]) #MCH
  #plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 4
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images): #for next batch use range(20,40); range(40,60) and so on
  plt.subplot(num_rows, 2*num_cols, 2*i+1) #for next batch use 2*(i-20)+1; 2*(i-40)+1 and so on
  plot_image(i, predictions[i], test_labels, test_images)
  plt.ylabel(i)
  plt.subplot(num_rows, 2*num_cols, 2*i+2) #for next batch use 2*(i-20)+2; 2*(i-40)+2 and so on
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()